Use the precalculated output from resnet50 and train a Dense (FC) model with two outputs

1. class of the fish (task of the problem)
2. boxes of the fish

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bcolz

from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications

In [3]:
def load_array(fname):
    return bcolz.open(fname)[:]

In [4]:
train_conv_layers = load_array('./data/resnet50/train_conv.arr')
train_labels = load_array('./data/resnet50/train_labels.arr')
train_borders = load_array('./data/resnet50/train_borders.arr')
valid_conv_layers = load_array('./data/resnet50/valid_conv.arr')
valid_labels = load_array('./data/resnet50/valid_labels.arr')
valid_borders = load_array('./data/resnet50/valid_borders.arr')

In [5]:
train_conv_layers.shape, train_labels.shape, train_borders.shape

((3025, 2048), (3025, 8), (3025, 4))

In [6]:
valid_conv_layers.shape, valid_labels.shape, valid_borders.shape

((752, 2048), (752, 8), (752, 4))

In [29]:
inp = layers.Input(shape=(2048,), dtype='float32')

In [30]:
p = 0.6
x = layers.Dense(512, activation='relu')(inp)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p / 2)(x)
x_bb = layers.Dense(4, name='bb')(x)
x_class = layers.Dense(8, activation='softmax', name='class')(x)

In [31]:
model = models.Model(inp, [x_class, x_bb])

In [32]:
model.compile(optimizers.Adam(lr=0.001), loss=['categorical_crossentropy', 'mse'], metrics=['accuracy'], loss_weights=[1., 0.001])

In [ ]:
model.fit(train_conv_layers, [train_labels, train_borders], 
          validation_data=(valid_conv_layers, [valid_labels, valid_borders]), 
          batch_size=64, epochs=5) 

After 15-20 epochs we got `loss: 0.5580 - class_loss: 0.1346 - bb_loss: 423.3748 - class_acc: 0.9570 - bb_acc: 0.7504 - val_loss: 0.7270 - val_class_loss: 0.3077 - val_bb_loss: 419.2943 - val_class_acc: 0.9269 - val_bb_acc: 0.7447`

## Submission

In [36]:
import submission

In [37]:
test_conv_layers = load_array('./data/resnet50/test_conv.arr')

In [38]:
test_filenames = load_array('./data/resnet50/test_filenames.arr')

In [39]:
preds = model.predict(test_conv_layers)

In [45]:
submission.gen_file(preds=preds[0], filenames=test_filenames, clip=True)

This gave a score of :
- public: 1.42140
- private: 2.35819

In general a little better than the just Dense model since is exactly the same model with two outputs instead of one.